In [ ]:
!pip list

In [1]:
pip install transformers datasets sacrebleu


ERROR: Could not find a version that satisfies the requirement sacrebleu (from versions: none)
ERROR: No matching distribution found for sacrebleu
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

# List the files in the directory
model_path = "/kaggle/input/pretrained-filess/"
files = os.listdir(model_path)
print(files)


['config.json', 'tokenizer_config.json', 'pytorch_model.bin', 'README (2).md', 'vocab.txt']


In [4]:
from transformers import BertForSequenceClassification, BertTokenizer, pipeline

# Specify the path where the model files are located
model_path = "/kaggle/input/pretrained-filess/"  # Path to the folder containing the model files

# Load the model and tokenizer from the local directory
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

# Specify the device (use 0 for GPU or -1 for CPU)
device = 0  # Set to 0 for GPU, -1 for CPU

# Create a text classification pipeline with the specified device
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device)

# Example SMS messages
sms = [
    "Congratulations! You've won a $1,000 Walmart gift card. Go to http://spam.com to claim now.",
    "Hi, are we still on for dinner tomorrow night?"
]

# Label mapping
label_map = {
    "LABEL_0": "spam",
    "LABEL_1": "ham"
}

# Get predictions
predictions = classifier(sms)

# Print predictions with human-readable labels
for i, message in enumerate(sms):
    predicted_label = label_map[predictions[i]["label"]]
    score = predictions[i]["score"]
    print(f"Message: {message}")
    print(f"Prediction: {predicted_label} (confidence: {score:.2f})\n")


Message: Congratulations! You've won a $1,000 Walmart gift card. Go to http://spam.com to claim now.
Prediction: spam (confidence: 0.79)

Message: Hi, are we still on for dinner tomorrow night?
Prediction: spam (confidence: 0.94)



In [4]:
pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.9 MB/s eta 0:00:0000:010:01
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.30
    Uninstalling SQLAlchemy-2.0.30:
      Successfully uninstalled SQLAlchemy-2.0.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.
Note:

In [5]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# 1. Load Dataset
data_path = "/kaggle/input/cleaned/cleaned_Spam_SMS.csv"
df = pd.read_csv(data_path)

# Preprocess dataset
texts = df["Message"].tolist()
labels = [1 if label == "spam" else 0 for label in df["Class"].tolist()]  # Spam = 1, Ham = 0

# Split into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

# 2. Tokenize Dataset
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128)

train_encodings = preprocess_function(train_texts)
val_encodings = preprocess_function(val_texts)

# 3. Create Dataset Class
class SpamDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

# Create datasets
train_dataset = SpamDataset(train_encodings, train_labels)
val_dataset = SpamDataset(val_encodings, val_labels)

# 4. Load Pre-trained BERT Model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# 5. Define Evaluation Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1).numpy()
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# 6. Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",           # Directory to save model checkpoints
    evaluation_strategy="epoch",     # Evaluate after each epoch
    save_strategy="epoch",           # Save model after each epoch
    learning_rate=5e-5,              # Learning rate
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    num_train_epochs=3,              # Number of epochs
    weight_decay=0.01,               # Weight decay for regularization
    logging_dir="./logs",            # Directory to save logs
    logging_steps=10,                # Log every 10 steps
    load_best_model_at_end=True,     # Load best model at the end of training
    metric_for_best_model="f1",      # Metric to determine the best model
    save_total_limit=2,              # Keep only 2 checkpoints
    report_to="none",                # Disable W&B or other integrations
)

# 7. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 8. Train and Save the Model
trainer.train()
trainer.save_model("./spam_classifier_model")
tokenizer.save_pretrained("./spam_classifier_model")

# 9. Evaluate Model
metrics = trainer.evaluate()
print("Evaluation Metrics:", metrics)

# 10. Test the Model on New Examples
# 10. Test the Model on New Examples
test_sms = [
    "Congratulations! You've won a $1,000 Walmart gift card. Go to http://spam.com to claim now.",
    "Hi, are we still on for dinner tomorrow night?",
]

# Tokenize the test messages
test_encodings = tokenizer(test_sms, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

# Move the encodings and model to the same device (GPU or CPU)
device = model.device  # Get the device of the model (GPU or CPU)
test_encodings = {key: val.to(device) for key, val in test_encodings.items()}  # Move inputs to the same device

# Make predictions
output = model(**test_encodings)
predictions = torch.argmax(output.logits, dim=-1).tolist()

# Map predictions to labels
label_map = {0: "ham", 1: "spam"}
for i, sms in enumerate(test_sms):
    print(f"Message: {sms}")
    print(f"Prediction: {label_map[predictions[i]]}")



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead uns

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.048000,0.031899,0.994619,0.987421,0.975155,0.981250
2,0.026700,0.017980,0.997309,0.993750,0.987578,0.990654
3,0.000500,0.016966,0.997309,0.993750,0.987578,0.990654


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

Evaluation Metrics: {'eval_loss': 0.017980312928557396, 'eval_accuracy': 0.9973094170403587, 'eval_precision': 0.99375, 'eval_recall': 0.9875776397515528, 'eval_f1': 0.9906542056074766, 'eval_runtime': 5.8201, 'eval_samples_per_second': 191.578, 'eval_steps_per_second': 6.014, 'epoch': 3.0}
Message: Congratulations! You've won a $1,000 Walmart gift card. Go to http://spam.com to claim now.
Prediction: spam
Message: Hi, are we still on for dinner tomorrow night?
Prediction: ham


In [6]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the pre-trained model and tokenizer
model = BertForSequenceClassification.from_pretrained("./spam_classifier_model")
tokenizer = BertTokenizer.from_pretrained("./spam_classifier_model")

# Function to make a prediction on a custom input
def predict_spam(input_texts):
    # Tokenize the input text
    encodings = tokenizer(input_texts, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    
    # Run the model on the input text
    with torch.no_grad():
        outputs = model(**encodings)
    
    # Get the predictions
    predictions = torch.argmax(outputs.logits, dim=-1).tolist()
    
    # Map predictions to labels
    label_map = {0: "ham", 1: "spam"}
    for i, text in enumerate(input_texts):
        print(f"Message: {text}")
        print(f"Prediction: {label_map[predictions[i]]}")

# Example usage with custom input
custom_texts = [
    "You've won a free vacation! Claim your prize now.",
    "Congratulation on your win today at the match"
]

predict_spam(custom_texts)



Message: You've won a free vacation! Claim your prize now.
Prediction: ham
Message: Congratulation on your win today at the match
Prediction: ham


In [6]:
pip install gradio transformers torch


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 31.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 97.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: python-multipart
    Found existing installation: python-multipart 0.0.9
    Uninstalling python-multipart-0.0.9:
      Successfully uninstalled python-multipart-0.0.9
  Attempting uninstall: starlette
    Found existing installation: starlette 0.37.2
    Uninstalling starlette-0.37.2:
      Successfully uninstalled starlette-0.37.2
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.111.0
    Uninstalling fastapi-0.111.0:
      Successfully uninstalled fastapi-0.111.0
Note: you may need to restart the kernel to use updated package

In [7]:
import gradio as gr
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the pre-trained model and tokenizer
model = BertForSequenceClassification.from_pretrained("./spam_classifier_model")
tokenizer = BertTokenizer.from_pretrained("./spam_classifier_model")

# Define the prediction function
def classify_spam(input_text):
    # Tokenize the input text
    encodings = tokenizer(input_text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    
    # Run the model on the input text
    with torch.no_grad():
        outputs = model(**encodings)
    
    # Get the predictions and confidence (softmax output)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    confidence, prediction = torch.max(probabilities, dim=-1)
    
    # Map predictions to labels
    label_map = {0: "Not spam", 1: "Spam"}
    result = label_map[prediction.item()]
    
    # Return result with confidence percentage
    return f"{result} (Confidence: {confidence.item() * 100:.2f}%)"

# Create Gradio interface
iface = gr.Interface(
    fn=classify_spam,  # Function to be called
    inputs=gr.Textbox(label="Enter your message"),  # Input component for user to type message
    outputs=gr.Textbox(label="Prediction Result"),  # Output component to show prediction
    live=True,  # Update output in real-time
    title="Spam Message Classifier",  # Title of the app
    description="Enter a message to check if it's spam or not along with the confidence score.",  # Description
)

# Launch the app
iface.launch()


* Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://b7bbcfd941cc4907f8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
